In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
data=pd.read_parquet('raw_hos_6hour.parquet')
data

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [3]:
len(data[data['label']==1]['hadm_id'].unique())

928

In [4]:
928/2

464.0

# lstm_ED 데이터 train, val, test, val test

In [5]:
hadm_ids_ab_label = data[data['label'] == 1]['hadm_id'].unique()
print(f"abnormal_hadm_id_num:{len(hadm_ids_ab_label)}")

val_ab_test_id = np.random.choice(hadm_ids_ab_label, size=464, replace=False)
print(f"ab_val_test_id:{len(val_ab_test_id)}")

ab_test_id = np.setdiff1d(hadm_ids_ab_label, val_ab_test_id)
print(f"ab_test_id:{len(ab_test_id)}")

set(val_ab_test_id)&set(ab_test_id)

abnormal_hadm_id_num:928
ab_val_test_id:464
ab_test_id:464


set()

In [6]:
hadm_id_live_label = data[data['label'] == 0]['hadm_id'].unique()
print(f"normal_hadm_id_num:{len(hadm_id_live_label)}")

normal_test_id = np.random.choice(hadm_id_live_label, size=928, replace=False)
print(f"normal_test_id:{len(normal_test_id)}")

val_nor_test_id = np.random.choice(normal_test_id, size=464, replace=False)
print(f"nor_val_test_id:{len(val_nor_test_id)}")

nor_test_id = np.setdiff1d(normal_test_id, val_nor_test_id)
print(f"nor_test_id:{len(nor_test_id)}")

real_normal = np.setdiff1d(hadm_id_live_label, normal_test_id)
print(f"real_normal:{len(real_normal)}")

train_id = np.random.choice(real_normal, size=round(len(real_normal)*0.9), replace=False)
print(f"train_id:{len(train_id)}")

val_id = np.setdiff1d(real_normal, train_id)
print(f"val_id:{len(val_id)}")

set(train_id)&set(val_id)&set(val_nor_test_id)&set(nor_test_id)

normal_hadm_id_num:48333
normal_test_id:928
nor_val_test_id:464
nor_test_id:464
real_normal:47405
train_id:42664
val_id:4741


set()

In [7]:
print(f"train_id:{len(train_id)}")
print(f"val_id:{len(val_id)}")

val_test_id=set(val_nor_test_id)|set(val_ab_test_id)
test_id=set(nor_test_id)|set(ab_test_id)

print(f"val_test_id:{len(val_test_id)}")
print(f"test_id:{len(test_id)}")

train_id:42664
val_id:4741
val_test_id:928
test_id:928


In [8]:
columns_to_drop = ['label']
result_df_4 = data.drop(columns=columns_to_drop)

In [9]:
result_df_4

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [48]:
#columns_to_drop = ['result']
#result_df_2 = result_df_1.drop(columns=columns_to_drop)

In [10]:
train_df=result_df_4[result_df_4['hadm_id'].isin(train_id)]

val_df=result_df_4[result_df_4['hadm_id'].isin(val_id)]

val_test_df=result_df_4[result_df_4['hadm_id'].isin(val_test_id)]

test_df=result_df_4[result_df_4['hadm_id'].isin(test_id)]

In [11]:
train_df.reset_index(inplace=True,drop=True)
val_df.reset_index(inplace=True,drop=True)
val_test_df.reset_index(inplace=True,drop=True)
test_df.reset_index(inplace=True,drop=True)

# scaler

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler1 = MinMaxScaler()
#scaler2 = StandardScaler()
#columns_to_scale = train_x.columns[2:].tolist()
one_hot = train_df.columns[0:2].tolist()#+train_df.columns[20:58].tolist()+train_df.columns[59:105].tolist()+train_df.columns[111:].tolist()
var = [col for col in train_df.columns if col not in one_hot]

train_x = train_df.copy()  
train_x[var] = scaler1.fit_transform(train_x[var]) #scaler1 할땐 바꿔줘야함

val_x = val_df.copy()
val_test_x = val_test_df.copy()
test_x = test_df.copy()

val_x[var] = scaler1.transform(val_x[var])
val_test_x[var] = scaler1.transform(val_test_x[var])
test_x[var] = scaler1.transform(test_x[var])

In [13]:
train_df

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [39]:
train_df.columns[0:40].tolist()+train_df.columns[194:346].tolist()+train_df.columns[347:].tolist()

['hadm_id',
 'time',
 'curr_service_CMED',
 'curr_service_CSURG',
 'curr_service_DENT',
 'curr_service_ENT',
 'curr_service_EYE',
 'curr_service_GU',
 'curr_service_GYN',
 'curr_service_MED',
 'curr_service_NMED',
 'curr_service_NSURG',
 'curr_service_OBS',
 'curr_service_OMED',
 'curr_service_ORTHO',
 'curr_service_PSURG',
 'curr_service_PSYCH',
 'curr_service_SURG',
 'curr_service_TRAUM',
 'curr_service_TSURG',
 'curr_service_VSURG',
 'drug_acetaminophen',
 'drug_aspirin',
 'drug_bisacodyl',
 'drug_docusatesodium',
 'drug_furosemide',
 'drug_heparin',
 'drug_hydromorphone(dilaudid)',
 'drug_insulin',
 'drug_lorazepam',
 'drug_magnesiumsulfate',
 'drug_metoprololtartrate',
 'drug_morphinesulfate',
 'drug_ondansetron',
 'drug_oxycodone(immediaterelease)',
 'drug_pantoprazole',
 'drug_potassiumchloride',
 'drug_senna',
 'drug_sodiumchloride0.9%flush',
 'drug_vancomycin',
 'inpute_max_Eosinophils',
 'inpute_max_HCT',
 'inpute_max_hemoglobin',
 'inpute_max_Lymphocytes',
 'inpute_max_MCH',

In [40]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
#columns_to_scale = train_x.columns[2:].tolist()
one_hot = train_df.columns[0:40].tolist()+train_df.columns[194:346].tolist()+train_df.columns[347:].tolist()
var = [col for col in train_df.columns if col not in one_hot]

train_x = train_df.copy()  
train_x[var] = scaler2.fit_transform(train_x[var]) #scaler1 할땐 바꿔줘야함

val_x = val_df.copy()
val_test_x = val_test_df.copy()
test_x = test_df.copy()

val_x[var] = scaler2.transform(val_x[var])
val_test_x[var] = scaler2.transform(val_test_x[var])
test_x[var] = scaler2.transform(test_x[var])

In [42]:
train_x['hadm_id'].value_counts()

hadm_id
21659432.0    83
28413592.0    83
26494665.0    83
29855554.0    83
28089339.0    83
              ..
20967163.0     1
28316602.0     1
25305994.0     1
24045599.0     1
23376243.0     1
Name: count, Length: 42664, dtype: int64

In [50]:
def pad_sequences(dataframe, padding_label, sequence_length=83):
    hadm_ids = dataframe['hadm_id'].unique()
    num_vars = dataframe.shape[1]
    padded_data = []
    
    for hadm_id in tqdm(hadm_ids):
        group_data = dataframe[dataframe['hadm_id'] == hadm_id].iloc[:, :].values
        num_rows = group_data.shape[0]
        
        if num_rows < sequence_length:
            padding_needed = sequence_length - num_rows
            padding_array = np.full((padding_needed, num_vars), padding_label)
            padded_group_data = np.concatenate((group_data, padding_array))
        else:
            padded_group_data = group_data
        
        padded_data.append(padded_group_data)
    
    padded_array = np.concatenate(padded_data)
    padded_df = pd.DataFrame(padded_array, columns=dataframe.columns.tolist())
    padded_df['hadm_id'].replace(0, np.nan, inplace=True)
    padded_df['hadm_id'].fillna(method='ffill', inplace=True)
    
    return padded_df

In [51]:
train_x_padded = pad_sequences(train_x, 0) #train_df, train_x
val_x_padded = pad_sequences(val_x, 0)
val_test_x_padded = pad_sequences(val_test_x, 0)
test_x_padded = pad_sequences(test_x, 0)

100%|██████████| 42664/42664 [01:46<00:00, 400.74it/s]
C:\Users\DAHS\AppData\Local\Temp\ipykernel_21308\1841793871.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  padded_df['hadm_id'].fillna(method='ffill', inplace=True)
100%|██████████| 4741/4741 [00:09<00:00, 495.63it/s]
C:\Users\DAHS\AppData\Local\Temp\ipykernel_21308\1841793871.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  padded_df['hadm_id'].fillna(method='ffill', inplace=True)
100%|██████████| 928/928 [00:02<00:00, 456.27it/s]
C:\Users\DAHS\AppData\Local\Temp\ipykernel_21308\1841793871.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  padded_df['hadm_id'].fillna(method='ffill', inplace=True)
100%|██████████| 928/928 [00:01<00:00, 528.66it/s]
C:\Users\D

# test 데이터 전처리

In [52]:
dfs=[val_test_x_padded, test_x_padded]

In [53]:
val_test_x_padded.shape[1]

385

In [55]:
val_test_x_padded

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [57]:
max_seq_len = 83
n_col = val_test_x_padded.shape[1]
new_dfs = []
for df in dfs:
    tmp = df.values.reshape(-1, max_seq_len, n_col)
    mask_padded = tmp[:, :, 1] == 0
    len_seqs = 83-np.sum(mask_padded, axis=1)
    seqs = []

    for idx, ls in tqdm(enumerate(len_seqs)):
        if ls == 1:
            _seq = tmp[idx].copy()
            seqs.append(_seq.copy())
            continue
        
        hadm_id = tmp[idx, 0, 0]
        result = tmp[idx, 0, -1]
        
        for i in range(1, ls):
            _seq = tmp[idx].copy()
            _seq[i:, :] = [[hadm_id, '0']+np.repeat(0, n_col-3).tolist()+[result] for _ in range(max_seq_len-i)]
            seqs.append(_seq.copy())
        
        _seq = tmp[idx].copy()
        seqs.append(_seq.copy())

    seqs = np.array(seqs)
    seqs = seqs.reshape(-1, n_col)

    new_dfs.append(pd.DataFrame(seqs, columns=df.columns))

928it [00:13, 69.11it/s]
928it [00:14, 65.50it/s]


In [58]:
train_x_padded

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [61]:
df_std_hos_dict_6hour = {
    "train_x": train_x_padded,
    "val_x": val_x_padded,
    "val_test_x": new_dfs[0],
    "test_x": new_dfs[1],
}

In [62]:
import pickle

with open(f'df_std_hos_dict_6hour.pkl', 'wb') as f:
    pickle.dump(df_std_hos_dict_6hour, f)